In [6]:
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import BaseTool, StructuredTool, tool
from dotenv import load_dotenv
import os
load_dotenv()

In [ ]:
EMAIL=os.getenv('EMAIL')
PASSWORD=os.getenv('PASSWORD')

In [7]:
from typing import Optional, Type
from email.message import EmailMessage
import smtplib
import ssl

from langchain.callbacks.manager import (
    AsyncCallbackManagerForToolRun,
    CallbackManagerForToolRun,
)


class SendMailInput(BaseModel):
    receiver: str = Field(description="receiver of the email")
    subject: str = Field(description="subject of the email")
    body: str = Field(description="body of the email")


class SendEmailTool(BaseTool):
    name = "email"
    description = "useful for when you need to send email"
    args_schema: Type[BaseModel] = SendMailInput

    def _run(
        self, receiver: str,subject: str ,body: str ,run_manager: Optional[CallbackManagerForToolRun] = None
    ) -> str:
        """Use the tool."""
        email_sender = EMAIL
        email_password = PASSWORD

        em = EmailMessage()
        em['From'] = email_sender
        em['To'] = receiver
        em['Subject'] = subject
        em.set_content(body)

        context = ssl.create_default_context()

        with smtplib.SMTP_SSL('smtp.gmail.com', 465,context=context) as smtp:
            smtp.login(email_sender, email_password)
            smtp.sendmail(email_sender, receiver, em.as_string())
        
        return "sent"

    async def _arun(
        self, query: str, run_manager: Optional[AsyncCallbackManagerForToolRun] = None
    ) -> str:
        """Use the tool asynchronously."""
        raise NotImplementedError("custom_search does not support async")



In [8]:
emailTool = SendEmailTool()

In [9]:
emailTool.run({"receiver": "tl@zasmlabs.com", "subject": "hello", "body": "test"})

'sent'

In [10]:
from langchain_core.tools import tool


@tool
def EmailTool(receiver: str, subject:str,body: str) -> int:
    """Sends and email to receiver with subject and body.

    Args:
        receiver: receiver of the mail
        subject: subject of the mail
        body: body of the mail
    """
    email_sender = EMAIL
    email_password = PASSWORD

    em = EmailMessage()
    em['From'] = email_sender
    em['To'] = receiver
    em['Subject'] = subject
    em.set_content(body)

    context = ssl.create_default_context()

    with smtplib.SMTP_SSL('smtp.gmail.com', 465,context=context) as smtp:
        smtp.login(email_sender, email_password)
        smtp.sendmail(email_sender, receiver, em.as_string())
        
    return "sent"


In [12]:
from langchain_google_vertexai import ChatVertexAI

In [13]:
import vertexai

vertexai.init(
    project="bim-poc",
    location="us-central1",
)
llm = ChatVertexAI(model="gemini-pro")

In [16]:
from langchain_core.messages import HumanMessage, ToolMessage

llm_with_tools = llm.bind_tools([EmailTool])


In [32]:
query = "Send an email to tl@zasmlabs.com regarding asking for a leave for 2 days keep it in details and give the reason for leave as family emergency"
messages = [HumanMessage(query)]


ai_msg = llm_with_tools.invoke(query)
messages.append(ai_msg)

for tool_call in ai_msg.tool_calls:
    selected_tool = {"EmailTool": EmailTool}[tool_call["name"]]
    tool_output = selected_tool.invoke(tool_call["args"])
    messages.append(ToolMessage(tool_output, tool_call_id=tool_call["id"]))

messages




[HumanMessage(content='Send an email to tl@zasmlabs.com regarding asking for a leave for 2 days keep it in details and give the reason for leave as family emergency'),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'EmailTool', 'arguments': '{"receiver": "tl@zasmlabs.com", "subject": "Leave of Absence Request", "body": "Dear Team Leader,\\\\n\\\\nI am writing to request a leave of absence for 2 days, from [Start Date] to [End Date], due to a family emergency.\\\\n\\\\nA family member has fallen ill and requires my immediate attention. I will be unavailable during this time to attend to their needs.\\\\n\\\\nI have completed all my urgent tasks and made arrangements for my colleagues to cover my responsibilities while I am away. I will also be available remotely if needed.\\\\n\\\\nI apologize for any inconvenience this may cause and appreciate your understanding in this matter.\\\\n\\\\nThank you,\\\\n[Your Name]"}'}}, response_metadata={'is_blocked': False, 'safet

In [1]:
!pip3 install -q langchain-fireworks

In [2]:
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_fireworks import ChatFireworks

In [4]:
chat = ChatFireworks(model="accounts/fireworks/models/firefunction-v2-rc",temperature=0)

In [14]:
chat_with_tools = chat.bind_tools([EmailTool])

In [17]:
query = "Send an email to tl@zasmlabs.com regarding asking for a leave for 2 days keep it in details and give the reason for leave as family emergency"
messages = [HumanMessage(query)]


ai_msg = chat_with_tools.invoke(query)
messages.append(ai_msg)

for tool_call in ai_msg.tool_calls:
    selected_tool = {"EmailTool": EmailTool}[tool_call["name"]]
    tool_output = selected_tool.invoke(tool_call["args"])
    messages.append(ToolMessage(tool_output, tool_call_id=tool_call["id"]))

messages


[HumanMessage(content='Send an email to tl@zasmlabs.com regarding asking for a leave for 2 days keep it in details and give the reason for leave as family emergency'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_U4EVd13VqC3VmzqRKohid6Rd', 'type': 'function', 'function': {'name': 'EmailTool', 'arguments': '{"receiver": "tl@zasmlabs.com", "subject": "Request for Leave for 2 Days", "body": "Dear TL,\\n\\nI am writing to request a leave for 2 days due to a family emergency. I apologize for any inconvenience this may cause and will make sure to catch up on any missed work as soon as possible.\\n\\nThank you for your understanding.\\n\\nBest regards,\\n[Your Name]"}'}}]}, response_metadata={'token_usage': {'prompt_tokens': 431, 'total_tokens': 537, 'completion_tokens': 106}, 'model_name': 'accounts/fireworks/models/firefunction-v2-rc', 'system_fingerprint': '', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-54184305-e075-45a1-b459-a405bfb90359

In [23]:
!pip3 install -qU langchain-groq

In [25]:
from langchain_groq import ChatGroq

groq_llm = ChatGroq(model = 'llama3-70b-8192')


In [26]:
groq_llm_tools = groq_llm.bind_tools([EmailTool])

In [27]:
query = "Send an email to tl@zasmlabs.com regarding asking for a half day for 2 days keep it in details and give the reason for leave as family emergency"
messages = [HumanMessage(query)]


ai_msg = chat_with_tools.invoke(query)
messages.append(ai_msg)

for tool_call in ai_msg.tool_calls:
    selected_tool = {"EmailTool": EmailTool}[tool_call["name"]]
    tool_output = selected_tool.invoke(tool_call["args"])
    messages.append(ToolMessage(tool_output, tool_call_id=tool_call["id"]))

messages


[HumanMessage(content='Send an email to tl@zasmlabs.com regarding asking for a half day for 2 days keep it in details and give the reason for leave as family emergency'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_9D3lmtWx4fvo8cucj1nolLmq', 'type': 'function', 'function': {'name': 'EmailTool', 'arguments': '{"receiver": "tl@zasmlabs.com", "subject": "Request for Half Day Leave for 2 Days", "body": "Dear TL,\\n\\nI am writing to request a half day leave for the next two days due to a family emergency that requires my immediate attention. I apologize for any inconvenience this may cause and will make sure to catch up on any missed work as soon as possible.\\n\\nThank you for your understanding and support.\\n\\nBest regards,\\n[Your Name]"}'}}]}, response_metadata={'token_usage': {'prompt_tokens': 432, 'total_tokens': 550, 'completion_tokens': 118}, 'model_name': 'accounts/fireworks/models/firefunction-v2-rc', 'system_fingerprint': '', 'finish_reaso